In [1]:
import pandas as pd
import numpy as np
import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
EXPERIMENT_NAME = 'P3856'
RUN_NAME = 'P3856_YHE211_1_Slot1-1_1_5104'

In [3]:
MZ_TOLERANCE_PPM = 5
MZ_TOLERANCE_PERCENT = MZ_TOLERANCE_PPM * 10**-4
SCAN_TOLERANCE = 2
RT_TOLERANCE = 1

##### 3DID parameters

In [4]:
# load the detected features
EXPERIMENT_DIR = '/data2/experiments/{}'.format(EXPERIMENT_NAME)
FEATURES_DIR = '{}/features-3did'.format(EXPERIMENT_DIR)
FEATURES_FILE = '{}/exp-{}-run-{}-features-3did.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
FEATURES_DEDUP_FILE = '{}/exp-{}-run-{}-features-3did-dedup.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
MZ_COLUMN_NAME = 'monoisotopic_mz'

In [5]:
features_df = pd.read_pickle(FEATURES_FILE)

##### PASEF parameters

In [6]:
print('there are {} features prior to de-dup'.format(len(features_df)))

there are 182811 features prior to de-dup


In [7]:
features_df.columns

Index(['monoisotopic_mz', 'charge', 'monoisotopic_mass', 'feature_intensity',
       'envelope_mono_peak_intensity', 'envelope', 'isotope_count',
       'deconvolution_score', 'precursor_id',
       'envelope_mono_peak_three_sigma_intensity', 'scan_apex', 'scan_lower',
       'scan_upper', 'rt_apex', 'rt_lower', 'rt_upper', 'feature_id'],
      dtype='object')

In [8]:
# features_df['mz'] = features_df.recalibrated_monoisotopic_mz  # shorthand to reduce verbosity
features_df['dup_mz'] = features_df[MZ_COLUMN_NAME]  # shorthand to reduce verbosity
features_df['dup_mz_ppm_tolerance'] = features_df.dup_mz * MZ_TOLERANCE_PERCENT / 100
features_df['dup_mz_lower'] = features_df.dup_mz - features_df.dup_mz_ppm_tolerance
features_df['dup_mz_upper'] = features_df.dup_mz + features_df.dup_mz_ppm_tolerance
features_df['dup_scan_lower'] = features_df.scan_apex - SCAN_TOLERANCE
features_df['dup_scan_upper'] = features_df.scan_apex + SCAN_TOLERANCE
features_df['dup_rt_lower'] = features_df.rt_apex - RT_TOLERANCE
features_df['dup_rt_upper'] = features_df.rt_apex + RT_TOLERANCE
# features_df['composite_key'] = features_df.apply(lambda row: '{},{},{}'.format(row.feature_id, row.precursor_id, row.percolator_idx), axis=1)
features_df['dup_composite_key'] = features_df.apply(lambda row: '{},{}'.format(row.feature_id, row.precursor_id), axis=1)

In [9]:
# remove these after we're finished
columns_to_drop_l = []
columns_to_drop_l.append('dup_mz')
columns_to_drop_l.append('dup_mz_ppm_tolerance')
columns_to_drop_l.append('dup_mz_lower')
columns_to_drop_l.append('dup_mz_upper')
columns_to_drop_l.append('dup_scan_lower')
columns_to_drop_l.append('dup_scan_upper')
columns_to_drop_l.append('dup_rt_lower')
columns_to_drop_l.append('dup_rt_upper')
columns_to_drop_l.append('dup_composite_key')


In [10]:
features_df.sort_values(by=['feature_intensity'], ascending=False, inplace=True)

In [11]:
# see if any detections have a duplicate - if so, find the dup with the highest intensity and keep it
keep_l = []
for row in features_df.itertuples():
    dup_df = features_df[(features_df.dup_mz > row.dup_mz_lower) & (features_df.dup_mz < row.dup_mz_upper) & (features_df.scan_apex > row.dup_scan_lower) & (features_df.scan_apex < row.dup_scan_upper) & (features_df.rt_apex > row.dup_rt_lower) & (features_df.rt_apex < row.dup_rt_upper)].copy()
    # group the dups by charge - take the most intense for each charge
    for group_name,group_df in dup_df.groupby(['charge'], as_index=False):
        keep_l.append(group_df.iloc[0].dup_composite_key)


In [12]:
# remove any features that are not in the keep list
dedup_df = features_df[features_df.dup_composite_key.isin(keep_l)].copy()

In [13]:
number_of_dups = len(features_df)-len(dedup_df)
print('removed {} duplicates ({}% of the original detections)'.format(number_of_dups, round(number_of_dups/len(features_df)*100)))
print('there are {} detected de-duplicated features'.format(len(dedup_df)))

removed 21 duplicates (0% of the original detections)
there are 182790 detected de-duplicated features


In [14]:
# remove the columns we added earlier
dedup_df.drop(columns_to_drop_l, axis=1, inplace=True)

In [15]:
dedup_df.sample(n=3)

,monoisotopic_mz,charge,monoisotopic_mass,feature_intensity,envelope_mono_peak_intensity,envelope,isotope_count,deconvolution_score,precursor_id,envelope_mono_peak_three_sigma_intensity,scan_apex,scan_lower,scan_upper,rt_apex,rt_lower,rt_upper,feature_id
2,717.965117,5.0,3584.789207,18464.0,5957.0,"[[718.3664594144516, 5957.0], [718.36645941445...",6.0,61.925594,12249.0,4766.0,553.0,536.0,567.0,2042.616955,2038.384417,2047.380789,1224903
2,755.473012,4.0,3017.862946,5539.0,470.0,"[[755.5316067533754, 470.0], [755.723851164268...",6.0,39.708516,16140.0,75.0,615.0,594.0,627.0,1659.971843,1655.735556,1664.204048,1614003
2,992.963293,2.0,1983.912035,31117.0,9856.0,"[[992.9579628876895, 9856.0], [993.46497079988...",4.0,249.527360,49261.0,7647.0,270.0,270.0,278.0,1800.766571,1795.766571,1805.766571,4926103


In [16]:
dedup_df.to_pickle(FEATURES_DEDUP_FILE)